In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
%cd /content/drive/MyDrive/Colab Notebooks/intro ml

/content/drive/MyDrive/Colab Notebooks/intro ml


In [3]:
import os
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Data cleaning

## Kidney dataset

In [4]:
df = pd.read_csv('kidney_disease.csv')
df.head(10)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35,7300,4.6,no,no,no,good,no,no,ckd
5,5,60.0,90.0,1.015,3.0,0.0,NaN,NaN,notpresent,notpresent,74.0,25.0,1.1,142.0,3.2,12.2,39,7800,4.4,yes,yes,no,good,yes,no,ckd
6,6,68.0,70.0,1.010,0.0,0.0,NaN,normal,notpresent,notpresent,100.0,54.0,24.0,104.0,4.0,12.4,36,NaN,NaN,no,no,no,good,no,no,ckd
7,7,24.0,NaN,1.015,2.0,4.0,normal,abnormal,notpresent,notpresent,410.0,31.0,1.1,NaN,NaN,12.4,44,6900,5,no,yes,no,good,yes,no,ckd
8,8,52.0,100.0,1.015,3.0,0.0,normal,abnormal,present,notpresent,138.0,60.0,1.9,NaN,NaN,10.8,33,9600,4.0,yes,yes,no,good,no,yes,ckd
9,9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,107.0,7.2,114.0,3.7,9.5,29,12100,3.7,yes,yes,no,poor,no,yes,ckd


In [5]:
df.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [6]:
# Drop index column
y = df["id"]
df = df.drop(columns="id")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             391 non-null    float64
 1   bp              388 non-null    float64
 2   sg              353 non-null    float64
 3   al              354 non-null    float64
 4   su              351 non-null    float64
 5   rbc             248 non-null    object 
 6   pc              335 non-null    object 
 7   pcc             396 non-null    object 
 8   ba              396 non-null    object 
 9   bgr             356 non-null    float64
 10  bu              381 non-null    float64
 11  sc              383 non-null    float64
 12  sod             313 non-null    float64
 13  pot             312 non-null    float64
 14  hemo            348 non-null    float64
 15  pcv             330 non-null    object 
 16  wc              295 non-null    object 
 17  rc              270 non-null    obj

In [7]:
df.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,391,NaN,NaN,NaN,51.4834,17.1697,2,42,55,64.5,90
bp,388,NaN,NaN,NaN,76.4691,13.6836,50,70,80,80,180
sg,353,NaN,NaN,NaN,1.01741,0.00571662,1.005,1.01,1.02,1.02,1.025
al,354,NaN,NaN,NaN,1.01695,1.35268,0,0,0,2,5
su,351,NaN,NaN,NaN,0.450142,1.09919,0,0,0,0,5
rbc,248,2,normal,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pc,335,2,normal,259,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pcc,396,2,notpresent,354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ba,396,2,notpresent,374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bgr,356,NaN,NaN,NaN,148.037,79.2817,22,99,121,163,490


In [8]:
df.dtypes

age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv                object
wc                 object
rc                 object
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

For "object" datatype:

- pcv, wc and rc are in numerical type. 
- the others are categorical

In [9]:
nodataval = "\t?"
df = df.replace(nodataval, np.nan)

# Convert false string columns
other_numeric_columns = ["pcv", "wc", "rc"]
df[other_numeric_columns] = df[other_numeric_columns].apply(pd.to_numeric)

# Use categorical data type
is_str_cols = df.dtypes == object
str_columns = df.columns[is_str_cols]
categoric_columns = pd.Index(set(str_columns) - set(other_numeric_columns))
df[str_columns] = df[str_columns].astype("category")
df.dtypes

age                float64
bp                 float64
sg                 float64
al                 float64
su                 float64
rbc               category
pc                category
pcc               category
ba                category
bgr                float64
bu                 float64
sc                 float64
sod                float64
pot                float64
hemo               float64
pcv                float64
wc                 float64
rc                 float64
htn               category
dm                category
cad               category
appet             category
pe                category
ane               category
classification    category
dtype: object

In [10]:
# Check NaN percentage
missing_values = df.isna().sum(axis=0)
missing_count_pct = ((missing_values / len(df)) * 100)

print("Percentage of missing values corresponding to each column (unit=%)\n\n",missing_count_pct)

Percentage of missing values corresponding to each column (unit=%)

 age                2.25
bp                 3.00
sg                11.75
al                11.50
su                12.25
rbc               38.00
pc                16.25
pcc                1.00
ba                 1.00
bgr               11.00
bu                 4.75
sc                 4.25
sod               21.75
pot               22.00
hemo              13.00
pcv               17.75
wc                26.50
rc                32.75
htn                0.50
dm                 0.50
cad                0.50
appet              0.25
pe                 0.25
ane                0.25
classification     0.00
dtype: float64


For **numerical columns**, there are some that has **discrete values**, which means that they can just have some specific values, so we have to *replace their NaN with their max values*. 

They are: ***sg, al and su***

In [11]:
discrete_numerics = ["sg","al","su"]
fillna_mean_cols = pd.Index(
    set(df.columns[df.dtypes == "float64"]) - set(discrete_numerics))
fillna_most_cols = pd.Index(
    set(df.columns[df.dtypes == "category"]) | set(discrete_numerics))
assert set(fillna_mean_cols.union(fillna_most_cols)) == set(df.columns)

In [12]:
df[fillna_mean_cols] = df[fillna_mean_cols].fillna(df[fillna_mean_cols].mean())
df[fillna_most_cols] = df[fillna_most_cols].fillna(df[fillna_most_cols].mode().iloc[0])
df.head(10)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.000000,1.020,1.0,0.0,normal,normal,notpresent,notpresent,121.000000,36.0,1.2,137.528754,4.627244,15.4,44.0,7800.000000,5.200000,yes,yes,no,good,no,no,ckd
1,7.0,50.000000,1.020,4.0,0.0,normal,normal,notpresent,notpresent,148.036517,18.0,0.8,137.528754,4.627244,11.3,38.0,6000.000000,4.707435,no,no,no,good,no,no,ckd
2,62.0,80.000000,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.000000,53.0,1.8,137.528754,4.627244,9.6,31.0,7500.000000,4.707435,no,yes,no,poor,no,yes,ckd
3,48.0,70.000000,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.000000,56.0,3.8,111.000000,2.500000,11.2,32.0,6700.000000,3.900000,yes,no,no,poor,yes,yes,ckd
4,51.0,80.000000,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.000000,26.0,1.4,137.528754,4.627244,11.6,35.0,7300.000000,4.600000,no,no,no,good,no,no,ckd
5,60.0,90.000000,1.015,3.0,0.0,normal,normal,notpresent,notpresent,74.000000,25.0,1.1,142.000000,3.200000,12.2,39.0,7800.000000,4.400000,yes,yes,no,good,yes,no,ckd
6,68.0,70.000000,1.010,0.0,0.0,normal,normal,notpresent,notpresent,100.000000,54.0,24.0,104.000000,4.000000,12.4,36.0,8406.122449,4.707435,no,no,no,good,no,no,ckd
7,24.0,76.469072,1.015,2.0,4.0,normal,abnormal,notpresent,notpresent,410.000000,31.0,1.1,137.528754,4.627244,12.4,44.0,6900.000000,5.000000,no,yes,no,good,yes,no,ckd
8,52.0,100.000000,1.015,3.0,0.0,normal,abnormal,present,notpresent,138.000000,60.0,1.9,137.528754,4.627244,10.8,33.0,9600.000000,4.000000,yes,yes,no,good,no,yes,ckd
9,53.0,90.000000,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.000000,107.0,7.2,114.000000,3.700000,9.5,29.0,12100.000000,3.700000,yes,yes,no,poor,no,yes,ckd


## Banknote dataset

In [13]:
df = pd.read_csv('data_banknote_authentication.txt', header=None)
df.head(10)

,0,1,2,3,4
0,3.62160,8.6661,-2.80730,-0.44699,0
1,4.54590,8.1674,-2.45860,-1.46210,0
2,3.86600,-2.6383,1.92420,0.10645,0
3,3.45660,9.5228,-4.01120,-3.59440,0
4,0.32924,-4.4552,4.57180,-0.98880,0
5,4.36840,9.6718,-3.96060,-3.16250,0
6,3.59120,3.0129,0.72888,0.56421,0
7,2.09220,-6.8100,8.46360,-0.60216,0
8,3.20320,5.7588,-0.75345,-0.61251,0
9,1.53560,9.1772,-2.27180,-0.73535,0


In [14]:
df.columns = ["variance","skewness","curtosis","entropy","class"]

In [15]:
df.describe(include="all").transpose()

,count,mean,std,min,25%,50%,75%,max
variance,1372.0,0.433735,2.842763,-7.0421,-1.773000,0.49618,2.821475,6.8248
skewness,1372.0,1.922353,5.869047,-13.7731,-1.708200,2.31965,6.814625,12.9516
curtosis,1372.0,1.397627,4.310030,-5.2861,-1.574975,0.61663,3.179250,17.9274
entropy,1372.0,-1.191657,2.101013,-8.5482,-2.413450,-0.58665,0.394810,2.4495
class,1372.0,0.444606,0.497103,0.0000,0.000000,0.00000,1.000000,1.0000


In [16]:
df.dtypes

variance    float64
skewness    float64
curtosis    float64
entropy     float64
class         int64
dtype: object

In [17]:
df["class"] = df["class"].astype("category")

In [18]:
# Check NaN percentage
missing_values = df.isna().sum(axis=0)
missing_count_pct = ((missing_values / len(df)) * 100)

print("Percentage of missing values corresponding to each column (unit=%)\n\n",missing_count_pct)

Percentage of missing values corresponding to each column (unit=%)

 variance    0.0
skewness    0.0
curtosis    0.0
entropy     0.0
class       0.0
dtype: float64


*This dataset is already clean, no need much preprocessing*

In this part, I test the SVM classifier

In [165]:
from utils import *

In [44]:
def split_data_numpy(df_file, feature_list):
  """
  This function outputs numpy format
  Split between training set and test set
  Split the training set for cross-validation
  Inputs: 
    - df: 	
      type of pandas.core.frame.DataFrame
      cleaned data of Banknote Authentication Dataset or Chronic Kidney Disease

  Outputs:
    - train data:
    - valid data:
    - test data : 
  """
  dataset = df_file.to_numpy()
  # split ratio for training, testing and validation dataset
  testset_ratio = 0.2
  valiset_ratio = 0.1
  testset_length = int((dataset.shape[0])*testset_ratio)
  valiset_length = int((dataset.shape[0])*valiset_ratio)
  trainset_length = dataset.shape[0] - testset_length - valiset_length

  # Random shuffle dataset
  np.random.shuffle(dataset)
  # Split dataset
  X_train = dataset[:trainset_length, :-1]
  X_test = dataset[trainset_length:trainset_length + testset_length, :-1]
  Y_train = dataset[:trainset_length, -1]
  Y_test = dataset[trainset_length:trainset_length + testset_length, -1]
  return X_train, Y_train, X_test, Y_test

In [139]:
def category2numerical(df):
  cat_columns = df.select_dtypes(['category']).columns
  df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
  return df 

In [266]:
# infile = "data_banknote_authentication.txt"
infile = "kidney_disease.csv"
df = clean_data(infile)
#This is for kindney dataset only, with data_banknote, you can comment 
# this part between ### ###
###############
# df['classification'] = df['classification'].map({'ckd': 1, 'notckd': 0, 'ckd\t': 1})
# df['classification'] = df['classification'].astype("float64")
# There are still some unexpected symbols like \t or " ", I manually replace them.
df["classification"] = df["classification"].str.replace('\t',"")
df['classification'] = df['classification'].astype("category")
df["dm"] = df["dm"].str.replace('\t',"")
df["dm"] = df["dm"].str.replace(' ',"")
df["dm"] = df["dm"].astype("category")
# Normalize data before training
cont_features = []
for features in df.select_dtypes(include=['int64','float64']):
    cont_features.append(features)
# Scaling numerical features
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
df[cont_features] = minmax.fit_transform(df[cont_features].values)
# Change category to numerical
df = category2numerical(df)
###############
feature_list = ["variance","skewness","curtosis","entropy"] # So far I did not touch this one.
X_train, Y_train, X_test, Y_test = split_data_numpy(df, feature_list)

In [267]:
df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0.522727,0.230769,0.75,0.2,0.0,1,1,0,0,0.211538,0.088575,0.010582,0.839298,0.047803,0.836735,0.777778,0.231405,0.525424,1,1,1,0,0,0,0
1,0.056818,0.000000,0.75,0.8,0.0,1,1,0,0,0.269309,0.042362,0.005291,0.839298,0.047803,0.557823,0.644444,0.157025,0.441938,0,0,1,0,0,0,0
2,0.681818,0.230769,0.25,0.4,0.6,1,1,0,0,0.856838,0.132221,0.018519,0.839298,0.047803,0.442177,0.488889,0.219008,0.441938,0,1,1,1,0,1,0
3,0.522727,0.153846,0.00,0.8,0.0,1,0,1,0,0.202991,0.139923,0.044974,0.671924,0.000000,0.551020,0.511111,0.185950,0.305085,1,0,1,1,1,1,0
4,0.556818,0.230769,0.25,0.4,0.0,1,1,0,0,0.179487,0.062901,0.013228,0.839298,0.047803,0.578231,0.577778,0.210744,0.423729,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.602273,0.230769,0.75,0.0,0.0,1,1,0,0,0.252137,0.121951,0.001323,0.917981,0.053933,0.857143,0.844444,0.185950,0.474576,0,0,1,0,0,0,1
396,0.454545,0.153846,1.00,0.0,0.0,1,1,0,0,0.113248,0.075738,0.010582,0.861199,0.022472,0.911565,1.000000,0.231405,0.694915,0,0,1,0,0,0,1
397,0.113636,0.230769,0.75,0.0,0.0,1,1,0,0,0.166667,0.062901,0.002646,0.835962,0.042697,0.863946,0.888889,0.181818,0.559322,0,0,1,0,0,0,1
398,0.170455,0.076923,1.00,0.0,0.0,1,1,0,0,0.196581,0.124519,0.007937,0.823344,0.053933,0.755102,0.933333,0.206612,0.644068,0,0,1,0,0,0,1


In [270]:
# train
from sklearn import svm
clf = svm.SVC(kernel = "rbf", C = 5, gamma = 0.3)
clf.fit(X_train, Y_train)

SVC(C=5, gamma=0.3)

In [271]:
clf.predict(X_test), Y_test

(array([0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
        1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
        1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1.]),
 array([0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
        1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
        0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1.]))

In [272]:
# Print accuracy
error = clf.predict(X_test) - Y_test
accuracy = len(error[error == 0])/len(error)
print("Accuracy of the SVM model: ",accuracy)

Accuracy of the SVM model:  0.9875


**Python code for data clean up**

In [ ]:
path = "/content/drive/MyDrive/IMT 3eme annee/Intro to ML/Mini-project/data_banknote_authentication.txt"
import os
os.path.basename(path)

'data_banknote_authentication.txt'

In [ ]:
def clean_data(infile):
  # Input the path to the data file
  # To check type of columns, use df.dtypes | To check NaN value, use df.isna().sum(axis=0), where df = output cleaned dataset
  filename = os.path.basename(infile)
  if filename == "data_banknote_authentication.txt":
    df = pd.read_csv(filename, header=None)
    df.columns = ["variance","skewness","curtosis","entropy","class"]
    return df

  if filename == "kidney_disease.csv":
    df = pd.read_csv(filename)
    y = df["id"]
    df = df.drop(columns="id")

    # Handle "  ?" values
    nodataval = "\t?"
    df = df.replace(nodataval, np.nan)

    # Convert false string columns
    other_numeric_columns = ["pcv", "wc", "rc"]
    df[other_numeric_columns] = df[other_numeric_columns].apply(pd.to_numeric)

    # Use categorical data type
    is_str_cols = df.dtypes == object
    str_columns = df.columns[is_str_cols]
    categoric_columns = pd.Index(set(str_columns) - set(other_numeric_columns))
    df[str_columns] = df[str_columns].astype("category")

    # Handle NaN values
    # For numerical columns, there are some that has discrete values, which means that they can just have some specific values, 
    # so we have to replace their NaN with their max values. They are: sg, al and su
    discrete_numerics = ["sg","al","su"]
    fillna_mean_cols = pd.Index(
        set(df.columns[df.dtypes == "float64"]) - set(discrete_numerics))
    fillna_most_cols = pd.Index(
        set(df.columns[df.dtypes == "category"]) | set(discrete_numerics))
    assert set(fillna_mean_cols.union(fillna_most_cols)) == set(df.columns)
    df[fillna_mean_cols] = df[fillna_mean_cols].fillna(df[fillna_mean_cols].mean())
    df[fillna_most_cols] = df[fillna_most_cols].fillna(df[fillna_most_cols].mode().iloc[0])
    return df

  else:
    print("This is not the dataset we were to deal with. Please check its name to make sure it matches one of these two:\n", "data_banknote_authentication.txt\n", "kidney_disease.csv\n")

In [ ]:
file1 = "/content/drive/MyDrive/IMT 3eme annee/Intro to ML/Mini-project/kidney_disease.csv"
df = clean_data(file1)

In [ ]:
df.head(10)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.000000,1.020,1.0,0.0,normal,normal,notpresent,notpresent,121.000000,36.0,1.2,137.528754,4.627244,15.4,44.0,7800.000000,5.200000,yes,yes,no,good,no,no,ckd
1,7.0,50.000000,1.020,4.0,0.0,normal,normal,notpresent,notpresent,148.036517,18.0,0.8,137.528754,4.627244,11.3,38.0,6000.000000,4.707435,no,no,no,good,no,no,ckd
2,62.0,80.000000,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.000000,53.0,1.8,137.528754,4.627244,9.6,31.0,7500.000000,4.707435,no,yes,no,poor,no,yes,ckd
3,48.0,70.000000,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.000000,56.0,3.8,111.000000,2.500000,11.2,32.0,6700.000000,3.900000,yes,no,no,poor,yes,yes,ckd
4,51.0,80.000000,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.000000,26.0,1.4,137.528754,4.627244,11.6,35.0,7300.000000,4.600000,no,no,no,good,no,no,ckd
5,60.0,90.000000,1.015,3.0,0.0,normal,normal,notpresent,notpresent,74.000000,25.0,1.1,142.000000,3.200000,12.2,39.0,7800.000000,4.400000,yes,yes,no,good,yes,no,ckd
6,68.0,70.000000,1.010,0.0,0.0,normal,normal,notpresent,notpresent,100.000000,54.0,24.0,104.000000,4.000000,12.4,36.0,8406.122449,4.707435,no,no,no,good,no,no,ckd
7,24.0,76.469072,1.015,2.0,4.0,normal,abnormal,notpresent,notpresent,410.000000,31.0,1.1,137.528754,4.627244,12.4,44.0,6900.000000,5.000000,no,yes,no,good,yes,no,ckd
8,52.0,100.000000,1.015,3.0,0.0,normal,abnormal,present,notpresent,138.000000,60.0,1.9,137.528754,4.627244,10.8,33.0,9600.000000,4.000000,yes,yes,no,good,no,yes,ckd
9,53.0,90.000000,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.000000,107.0,7.2,114.000000,3.700000,9.5,29.0,12100.000000,3.700000,yes,yes,no,poor,no,yes,ckd


In [ ]:
df.isna().sum(axis=0)

age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64